# Simulación de la dinámica cuántica

Aplicar una serie de operadores unitarios (matrices) sobre un estado inicial |ψ⟩ y observar el estado final y su normalización.

En este notebook se muestra cómo usar una clase `QuantumDynamics` (si existe en tu paquete `quantum_simulator`) para aplicar una evolución por una lista de matrices unitarias. Si no está disponible, se proporciona una implementación alternativa directamente en la celda para que el notebook sea ejecutable de forma independiente.

## Importaciones y preparación

Importamos `QuantumDynamics` desde `quantum_simulator.dynamics`. Si la importación falla (ejemplo: por no tener ese paquete instalado en el `PYTHONPATH`), definimos una clase `QuantumDynamics` alternativa con la misma interfaz mínima que se usa abajo. Esto permite ejecutar el notebook sin dependencias externas.

In [ ]:
# Importaciones y fallback
import sys
sys.path.append('..')

import numpy as np

# Intentar importar la clase desde el paquete del proyecto
try:
    from quantum_simulator.dynamics import QuantumDynamics
    print("Importado QuantumDynamics desde quantum_simulator.dynamics")
except Exception as e:
    print("No se pudo importar quantum_simulator.dynamics. Se usará una implementación alternativa. Error:", e)
    class QuantumDynamics:
        """Implementación mínima de QuantumDynamics:
        - Inicializa con un estado (lista o array)
        - apply_unitary_evolution(lista_de_matrices) aplica las matrices en orden sobre el estado
        """
        def __init__(self, initial_state):
            a = np.asarray(initial_state, dtype=complex).reshape(-1)
            norm = np.linalg.norm(a)
            if norm == 0:
                raise ValueError("El estado inicial no puede ser el vector cero.")
            self.state = a / norm

        def apply_unitary_evolution(self, unitary_list):
            st = self.state.copy()
            for U in unitary_list:
                U = np.asarray(U, dtype=complex)
                # comprobación rápida de unitariedad (heurística)
                if U.shape[0] != U.shape[1]:
                    raise ValueError("Cada operador debe ser una matriz cuadrada.")
                # aplicar U sobre el estado
                st = U.dot(st)
            # actualizar y devolver
            self.state = st
            return self.state

## Dinámica con matrices unitarias

Definimos el estado inicial |0⟩ y una secuencia de operadores: primero la matriz de Hadamard H y luego la identidad I. Aplicamos la evolución en orden `[H, I]`.

Resultados mostrados:
- Estado inicial (como vector de amplitudes).
- Estado final tras aplicar las matrices.
- Norma del estado final (debe ser 1 si las matrices son unitarias y la implementación es correcta, salvo errores numéricos mínimos).

In [ ]:
# Dinámica con matrices unitarias
print("=== Dinamica del Sistema ===")
initial_state = [1, 0]  # Estado |0⟩

# Matriz de Hadamard
H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)], 
              [1/np.sqrt(2), -1/np.sqrt(2)]])

# Matriz identidad
I = np.eye(2)

dynamics = QuantumDynamics(initial_state)
final_state = dynamics.apply_unitary_evolution([H, I])

print(f"Estado inicial: {initial_state}")
print(f"Estado final (vector de amplitudes): {final_state}")
print(f"Norma del estado final: {np.linalg.norm(final_state):.4f}")

# Mostrar explicación corta de interpretación
print('\nInterpretación: La matriz de Hadamard transforma |0> en (|0> + |1>)/√2. Aplicar la identidad no cambia el estado. La norma debe mantenerse ~1.')

## Notas y Sugerencias

- Si quiere aplicar más operadores (por ejemplo rotaciones o puertas de 3x3), asegúrese de que sean unitarios y del mismo tamaño que el estado.
- Para usar su propia implementación `quantum_simulator.dynamics.QuantumDynamics`, ejecute el notebook desde la raíz del repositorio (o ajuste `sys.path.append()` con la ruta absoluta).
- Puede reemplazar la lista `[H, I]` por cualquier otra secuencia `[...]` para estudiar diferentes evoluciones.